In [1]:
!git clone https://github.com/eagle705/pytorch-bert-crf-ner.git

Cloning into 'pytorch-bert-crf-ner'...
remote: Enumerating objects: 1725, done.
remote: Counting objects: 100% (1725/1725), done.
remote: Compressing objects: 100% (1636/1636), done.
remote: Total 1725 (delta 118), reused 1662 (delta 71), pack-reused 0
Receiving objects: 100% (1725/1725), 27.16 MiB | 27.30 MiB/s, done.
Resolving deltas: 100% (118/118), done.


In [2]:
# 저장소로 이동
%cd pytorch-bert-crf-ner

/content/pytorch-bert-crf-ner


In [3]:
# !pip install -r requirements.txt

In [4]:
!pip install pytorch-crf 
!pip install pytorch-transformers==1.2.0
!pip install transformers==2.1.1  # https://github.com/SKTBrain/KoBERT/issues/31
!pip install torch==1.8.1
!pip install torchvision==0.9.1
!pip install gluonnlp
!pip install mxnet==1.8.0.post0
!pip install konlpy==0.5.2

     |████████████████████████████████| 176 kB 6.7 MB/s 
     |████████████████████████████████| 131 kB 55.3 MB/s 
     |████████████████████████████████| 895 kB 62.5 MB/s 
     |████████████████████████████████| 1.2 MB 58.3 MB/s 
     |████████████████████████████████| 79 kB 11.2 MB/s 
     |████████████████████████████████| 8.1 MB 18.8 MB/s 
     |████████████████████████████████| 138 kB 62.2 MB/s 
     |████████████████████████████████| 127 kB 72.4 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
     |████████████████████████████████| 311 kB 7.5 MB/s 
     |████████████████████████████████| 804.1 MB 2.6 kB

In [5]:
from __future__ import absolute_import, division, print_function, unicode_literals
import json
import pickle
import argparse
import torch
from kobert.pytorch_kobert import get_pytorch_kobert_model
from kobert.utils import get_tokenizer
from model.net import KobertSequenceFeatureExtractor, KobertCRF, KobertBiLSTMCRF, KobertBiGRUCRF
from gluonnlp.data import SentencepieceTokenizer
from data_utils.utils import Config
from data_utils.vocab_tokenizer import Tokenizer
from data_utils.pad_sequence import keras_pad_fn
from pathlib import Path

/usr/local/lib/python3.7/dist-packages/konlpy/tag/_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


## Train_Bert_CRF에서 학습 완료된 모델 불러오기

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
class DecoderFromNamedEntitySequence():
    def __init__(self, tokenizer, index_to_ner):
        self.tokenizer = tokenizer
        self.index_to_ner = index_to_ner

    def __call__(self, list_of_input_ids, list_of_pred_ids):
        input_token = self.tokenizer.decode_token_ids(list_of_input_ids)[0]
        pred_ner_tag = [self.index_to_ner[pred_id] for pred_id in list_of_pred_ids[0]]

        print("len: {}, input_token:{}".format(len(input_token), input_token))
        print("len: {}, pred_ner_tag:{}".format(len(pred_ner_tag), pred_ner_tag))

        # ----------------------------- parsing list_of_ner_word ----------------------------- #
        list_of_ner_word = []
        entity_word, entity_tag, prev_entity_tag = "", "", ""
        for i, pred_ner_tag_str in enumerate(pred_ner_tag):
            if "B-" in pred_ner_tag_str:
                entity_tag = pred_ner_tag_str[-3:]

                if prev_entity_tag != entity_tag and prev_entity_tag != "":
                    list_of_ner_word.append({"word": entity_word.replace("▁", " "), "tag": prev_entity_tag, "prob": None})

                entity_word = input_token[i]
                prev_entity_tag = entity_tag
            elif "I-"+entity_tag in pred_ner_tag_str:
                entity_word += input_token[i]
            else:
                if entity_word != "" and entity_tag != "":
                    list_of_ner_word.append({"word":entity_word.replace("▁", " "), "tag":entity_tag, "prob":None})
                entity_word, entity_tag, prev_entity_tag = "", "", ""


        # ----------------------------- parsing decoding_ner_sentence ----------------------------- #
        decoding_ner_sentence = ""
        is_prev_entity = False
        prev_entity_tag = ""
        is_there_B_before_I = False

        for token_str, pred_ner_tag_str in zip(input_token, pred_ner_tag):
            token_str = token_str.replace('▁', ' ')  # '▁' 토큰을 띄어쓰기로 교체

            if 'B-' in pred_ner_tag_str:
                if is_prev_entity is True:
                    decoding_ner_sentence += ':' + prev_entity_tag+ '>'

                if token_str[0] == ' ':
                    token_str = list(token_str)
                    token_str[0] = ' <'
                    token_str = ''.join(token_str)
                    decoding_ner_sentence += token_str
                else:
                    decoding_ner_sentence += '<' + token_str
                is_prev_entity = True
                prev_entity_tag = pred_ner_tag_str[-3:] # 첫번째 예측을 기준으로 하겠음
                is_there_B_before_I = True

            elif 'I-' in pred_ner_tag_str:
                decoding_ner_sentence += token_str

                if is_there_B_before_I is True: # I가 나오기전에 B가 있어야하도록 체크
                    is_prev_entity = True
            else:
                if is_prev_entity is True:
                    decoding_ner_sentence += ':' + prev_entity_tag+ '>' + token_str
                    is_prev_entity = False
                    is_there_B_before_I = False
                else:
                    decoding_ner_sentence += token_str

        return list_of_ner_word, decoding_ner_sentence

In [8]:
vocab_dir = Path('experiments/base_model_with_crf_val')

model_dir = Path('/content/drive/MyDrive/base_model_with_crf_val')
model_config = Config(json_path=model_dir / 'config.json')

# Vocab & Tokenizer
# tok_path = get_tokenizer() # ./tokenizer_78b3253a26.model
tok_path = "ptr_lm_model/tokenizer_78b3253a26.model"
ptr_tokenizer = SentencepieceTokenizer(tok_path)

# load vocab & tokenizer
with open(vocab_dir / "vocab.pkl", 'rb') as f:
    vocab = pickle.load(f)

tokenizer = Tokenizer(vocab=vocab, split_fn=ptr_tokenizer, pad_fn=keras_pad_fn, maxlen=model_config.maxlen)

# load ner_to_index.json
with open(model_dir / "ner_to_index.json", 'rb') as f:
    ner_to_index = json.load(f)
    index_to_ner = {v: k for k, v in ner_to_index.items()}

# Model
# model = KobertSequenceFeatureExtractor(config=model_config, num_classes=len(ner_to_index))
model = KobertCRF(config=model_config, num_classes=len(ner_to_index), vocab=vocab)
# model = KobertBiLSTMCRF(config=model_config, num_classes=len(ner_to_index), vocab=vocab)
# model = KobertBiGRUCRF(config=model_config, num_classes=len(ner_to_index), vocab=vocab)

# load
model_dict = model.state_dict()
# checkpoint = torch.load("./experiments/base_model/best-epoch-9-step-600-acc-0.845.bin", map_location=torch.device('cpu'))
# checkpoint = torch.load("./experiments/base_model_with_crf/best-epoch-16-step-1500-acc-0.993.bin", map_location=torch.device('cpu'))
checkpoint = torch.load("/content/drive/MyDrive/base_model_with_crf_val/best-epoch-12-step-1000-acc-0.960.bin", map_location=torch.device('cpu'))
# checkpoint = torch.load("./experiments/base_model_with_bilstm_crf/best-epoch-15-step-2750-acc-0.992.bin", map_location=torch.device('cpu'))
# checkpoint = torch.load("./experiments/base_model_with_bigru_crf/model-epoch-18-step-3250-acc-0.997.bin", map_location=torch.device('cpu'))

convert_keys = {}
for k, v in checkpoint['model_state_dict'].items():
    new_key_name = k.replace("module.", '')
    if new_key_name not in model_dict:
        print("{} is not int model_dict".format(new_key_name))
        continue
    convert_keys[new_key_name] = v

model.load_state_dict(convert_keys)
model.eval()
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# n_gpu = torch.cuda.device_count()
# if n_gpu > 1:
#     model = torch.nn.DataParallel(model)

decoder_from_res = DecoderFromNamedEntitySequence(tokenizer=tokenizer, index_to_ner=index_to_ner)

## 문장 별 개체명 인식 예측 함수

In [16]:
def NER_sentence(sentence):
    input_text = sentence
    list_of_input_ids = tokenizer.list_of_string_to_list_of_cls_sep_token_ids([input_text])
    x_input = torch.tensor(list_of_input_ids).long()

    ## for bert alone
    # y_pred = model(x_input)
    # list_of_pred_ids = y_pred.max(dim=-1)[1].tolist()

    ## for bert crf
    list_of_pred_ids = model(x_input)

    ## for bert bilstm crf & bert bigru crf
    # list_of_pred_ids = model(x_input, using_pack_sequence=False)

    list_of_ner_word, decoding_ner_sentence = decoder_from_res(list_of_input_ids=list_of_input_ids, list_of_pred_ids=list_of_pred_ids)
    
    return list_of_ner_word, decoding_ner_sentence

## 네이버 뉴스 기사 데이터 살펴보기

In [13]:
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/news_articles_ner.csv', header=None)
df.columns = ['date', 'category', 'broadcast', 'titles', 'sentences', 'link']
df.head()

,date,category,broadcast,titles,sentences,link
0,20170101,IT_science,YTN,2017년에 펼쳐지는 특별한 우주현상 2가지,앵커 새해 지구촌에는 특이한 우주 현상이 2차례 나타납니다. 4월에는 맨눈으로 볼 ...,https://news.naver.com/main/read.nhn?mode=LSD&...
1,20170101,IT_science,디지털데일리,DD 주간브리핑 사상 최대 규모로 열리는 CES 2017 에 관심 고조,2017년의 첫 주가 시작됐다. 새해 벽두 건강한 긴장감이 넘친다. 새해 첫 주부터...,https://news.naver.com/main/read.nhn?mode=LSD&...
2,20170101,IT_science,SBS,최첨단 미래TV 시대 열린다…어떤 서비스 있을까,앵커 올해는 UHD 차세대 초고화질 방송 시대가 열리는 첫해입니다. 시청자들이 어떤...,https://news.naver.com/main/read.nhn?mode=LSD&...
3,20170101,IT_science,세계일보,심층기획 2020년 우주기술 개발 자립국 대열 합류,세계 최초 인공위성 스푸트니크 발사 60주년 한국 우주개발 역사와 현주소 우주개발 ...,https://news.naver.com/main/read.nhn?mode=LSD&...
4,20170101,IT_science,세계일보,심층기획 지구 밖으로 쏘아 올린 인간의 꿈… 우주개발 무한경쟁,세계 최초 인공위성 스푸트니크 발사 60주년 우주 알리는 신호탄 1957년 10월4...,https://news.naver.com/main/read.nhn?mode=LSD&...


In [14]:
df.shape

(10, 6)

In [11]:
df['sentences'][0].split('. ')

['앵커 새해 지구촌에는 특이한 우주 현상이 2차례 나타납니다',
 '4월에는 맨눈으로 볼 수 있는 밝은 혜성이 찾아오고 8월에는 미 대륙을 횡단하는 개기일식 현상이 나타납니다',
 '정혜윤 기자의 보도입니다',
 '기자 하얗고 긴 꼬리를 드리우며 나타나는 혜성',
 '신비하고 아름답지만 맨눈으로 볼 수 있는 건 손으로 꼽을 정도로 드뭅니다',
 '그런데 올해 4월에는 북쪽 하늘에서 혜성을 관측할 수 있습니다',
 '5년여마다 한 번씩 지구를 찾는 41p 혜성입니다',
 '이태형 한국우주환경과학연구소장 41p 혜성은 4월 5일쯤 근지점을 통과하게 되는데 이때 거리는 지구로부터 달까지 거리의 20배 정도입니다',
 '최대밝기는 5등급 정도로 도심 밖에서는 맨눈으로 충분히 관측이 가능할 것으로 보입니다',
 '새해를 맞은 미국인은 8월의 행운을 기대하고 있습니다',
 '38년 만에 미 대륙을 서쪽에서 동쪽으로 가로지르는 개기일식 현상이 예고됐기 때문입니다',
 '개기 일식은 8월 21일 오전 미국 서부 오리건주에서 시작해 동쪽으로 이동하며 4시간여에 걸쳐 진행됩니다',
 '달 그림자 안에 들어가는 검은 색 지역에서는 개기일식이 그 주변에서는 부분일식이 나타납니다',
 '미 항공우주국 나사에 관련 정보가 올라오기 시작했고 미국인의 관심도 고조되고 있지만 아쉽게도 우리나라에서는 볼 수 없습니다',
 '우리나라에서 볼 수 있는 개기일식은 오는 2035년 9월에 예정돼 있습니다',
 'YTN 정혜윤입니다.']

In [40]:
df['sentences'][1].split('. ')

['2017년의 첫 주가 시작됐다',
 '새해 벽두 건강한 긴장감이 넘친다',
 '새해 첫 주부터 주목할만할 IT 현안이 많다',
 '먼저 세계 최대 가전 전시회인 CES 2017 행사에 국내 뿐만 아니라 전세계의 이목이 쏠려있다',
 '5일 현지시각 부터 4일간 미국 라스베이거스에서 진행된다',
 '특히 올해에는 역대 최다인 3800여개 업체가 참가하며 IT뿐만 아니라 자동차 인공지능 드론 등 관련 신제품도 대거 선보인다.']

In [12]:
df['sentences'][2].split('. ')

['앵커 올해는 UHD 차세대 초고화질 방송 시대가 열리는 첫해입니다',
 '시청자들이 어떤 방송 서비스를 즐길 수 있게 되는지 유병수 기자가 소개합니다',
 '기자 드라마와 스포츠 예능과 다큐멘터리 기존 HD 방송 보다 4배 이상 선명한 초고화질 UHD 방송입니다',
 '뛰어난 압축기술을 바탕으로 2개 이상의 채널은 물론 가상현실 VR 증강현실 AR 같은 차세대 방송 서비스를 즐길 수 있습니다',
 '또 지상파방송에 처음으로 인터넷 기반의 전송 기술이 접목돼 시청자가 직접 참여하는 최첨단 양방향 서비스도 경험할 수 있습니다',
 '심상민 성신여대 문화커뮤니케이션학부 교수 새로운 혁명적인 변화 4차산업혁명과 미디어의 새로운 진입 자체가 결합하는 신호탄이 바로 새로운 UHD TV의 서비스라 할 수 있습니다',
 '우리나라는 올해 차세대 초고화질 본방송을 시작해 내년 평창올림픽을 전 세계에 생중계합니다',
 '이를 위해 SBS는 지난해 새 방식의 초고화질 시험방송과 스포츠 생중계를 성공적으로 마쳤습니다',
 '이런 차세대 방송의 보편적 무료서비스를 위한 정부의 지원도 시급합니다',
 'UHD 프로그램 제작을 위해 지상파 방송사들이 앞으로 10년간 투자할 금액은 6조원에 육박할 것으로 예상됩니다',
 '재원 마련을 위해 유독 지상파 방송에만 부과된 정부 규제를 풀어야 한다는 지적이 많습니다',
 '한광석 남서울대 광고홍보학과 교수 UHD 제작비 자체가 1.5배에서 2배 정도 소요되기 때문에 중간광고와 같은 제도적인 부분을 완화해서 UHD 방송 콘텐츠 제작을 활성화할 필요가 있습니다',
 '차세대 초고화질 본방송은 올해 수도권에서 시작되고 내년 평창올림픽에 맞춰 강원도와 전국 광역시로 확대됩니다.']

## 예측

In [72]:
NER_word_result = {}
decoded_sentences = {}

for i in range(len(df)):
    sentences = df['sentences'][i].split('. ')
    NER_word_result[i] = {} ; decoded_sentences[i] = [] ; res= {}
    for idx, sent in enumerate(sentences):
        ner_word, decoded = NER_sentence(sent)

        if len(ner_word) != 0:
            for j in range(len(ner_word)):
                Tag = ner_word[j]['tag'] ; Word = ner_word[j]['word']
                res[Word] = Tag

        NER_word_result[i] = res
        decoded_sentences[i].append(decoded)  

len: 21, input_token:['[CLS]', '▁', '앵', '커', '▁새해', '▁지구', '촌', '에는', '▁특', '이', '한', '▁우주', '▁현상', '이', '▁2', '차례', '▁나', '타', '납', '니다', '[SEP]']
len: 21, pred_ner_tag:['[CLS]', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-NOH', 'I-NOH', 'O', 'O', 'O', 'O', '[SEP]']
len: 37, input_token:['[CLS]', '▁4', '월에는', '▁맨', '눈', '으로', '▁볼', '▁수', '▁있는', '▁밝은', '▁', '혜', '성이', '▁찾아', '오', '고', '▁8', '월에는', '▁미', '▁대', '륙', '을', '▁', '횡', '단', '하는', '▁개', '기', '일', '식', '▁현상', '이', '▁나', '타', '납', '니다', '[SEP]']
len: 37, pred_ner_tag:['[CLS]', 'B-DAT', 'I-DAT', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-DAT', 'I-DAT', 'B-LOC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', '[SEP]']
len: 9, input_token:['[CLS]', '▁정', '혜', '윤', '▁기', '자의', '▁보도', '입니다', '[SEP]']
len: 9, pred_ner_tag:['[CLS]', 'B-PER', 'I-PER', 'I-PER', 'O', 'O', 'O', 'O', '[SEP]']
len: 19, input_token:['[CLS]', '▁', '기자', '▁하', '[UNK]', '고', '

In [77]:
print(NER_word_result[0])
print(decoded_sentences[0])

{' 2차례': 'NOH', ' 4월에는': 'DAT', ' 8월에는': 'DAT', ' 미': 'LOC', ' 정혜윤': 'PER', ' 5년여': 'DUR', ' 한 번': 'NOH', ' 지구': 'LOC', ' 41p': 'NOH', ' 이태형': 'PER', ' 한국': 'ORG', ' 4월 5일': 'DAT', ' 달': 'LOC', ' 20배': 'PNT', ' 5등급': 'NOH', ' 8월': 'DAT', ' 38년': 'DUR', ' 8월 21일': 'DAT', ' 오리건주': 'LOC', ' 4시간': 'DUR', ' 나사': 'ORG', ' 2035년 9월에': 'DAT', ' YTN': 'ORG'}
['[CLS] 앵커 새해 지구촌에는 특이한 우주 현상이 <2차례:NOH> 나타납니다[SEP]', '[CLS] <4월에는:DAT> 맨눈으로 볼 수 있는 밝은 혜성이 찾아오고 <8월에는:DAT> <미:LOC> 대륙을 횡단하는 개기일식 현상이 나타납니다[SEP]', '[CLS] <정혜윤:PER> 기자의 보도입니다[SEP]', '[CLS] 기자 하[UNK]고 긴 꼬리를 드리우며 나타나는 혜성[SEP]', '[CLS] 신비하고 아름답지만 맨눈으로 볼 수 있는 건 손으로 꼽을 정도로 드[UNK]니다[SEP]', '[CLS] 그런데 올해 <4월에는:DAT> 북쪽 하늘에서 혜성을 관측할 수 있습니다[SEP]', '[CLS] <5년여:DUR>마다 <한 번:NOH>씩 <지구:LOC>를 찾는 <41p:NOH> 혜성입니다[SEP]', '[CLS] <이태형:PER> <한국:ORG>우주환경과학연구소장 <41p:NOH> 혜성은 <4월 5일:DAT>쯤 근지점을 통과하게 되는데 이때 거리는 <지구:LOC>로부터 <달:LOC>까지 거리의 <20배:PNT> 정도입니다[SEP]', '[CLS] 최대밝기는 <5등급:NOH> 정도로 도심 밖에서는 맨눈으로 충분히 관측이 가능할 것으로 보입니다[SEP]', '[CLS] 새해를 맞은 미국인은 <8월:DAT>의 행운을 기대하고 있습니다[

In [78]:
print(NER_word_result[1])
print(decoded_sentences[1])

{' 2017년': 'DAT', ' 첫': 'NOH', ' CES 2017': 'POH', ' 5일': 'DAT', ' 4일간': 'DUR', ' 라스베이거스': 'LOC', ' 3800여개': 'NOH'}
['[CLS] <2017년:DAT>의 <첫:NOH> 주가 시작됐다[SEP]', '[CLS] 새해 벽두 건강한 긴장감이 넘친다[SEP]', '[CLS] 새해 첫 주부터 주목할만할 IT 현안이 많다[SEP]', '[CLS] 먼저 세계 최대 가전 전시회인 <CES 2017:POH> 행사에 국내 뿐만 아니라 전세계의 이목이 쏠려있다[SEP]', '[CLS] <5일:DAT> 현지시각 부터 <4일간:DUR> <미국:LOC> <라스베이거스:LOC>에서 진행된다[SEP]', '[CLS] 특히 올해에는 역대 최다인 <3800여개:NOH> 업체가 참가하며 IT뿐만 아니라 자동차 인공지능 드론 등 관련 신제품도 대거 선보인다.[SEP]']
